# Manuel A. Rivas 
UK Biobank 

# Data access

Here we get access to UK Biobank data.

## Updated with 2019-2020 release
Let's generate the PLINK2 phenotype files for the groupings proposed by Greg Mcinnes. 

In [6]:
from __future__ import division
from __future__ import print_function
import pandas as pd 
import numpy as np

In [7]:

# read in mapping
highcf = open('high_confidence_matches_qcplus.tsv',encoding = "ISO-8859-1").readlines()
selfrcdictnew = {}
selfrckeysnew = {}
for line in highcf[0:]:
    line = line.rstrip()
    line = line.split('\t')
    selfrc = line[0]
    meaning = line[1]
    if int(selfrc) == -1:
        continue
    if len(line) != 5:
        if selfrc in selfrckeysnew:
            continue
        else:
            selfrckeysnew[selfrc] = meaning
            selfrcdictnew[selfrc,'meaning'] = meaning
            selfrcdictnew[selfrc,'icd'] = []
        continue
    icd10 = line[2] 
    icd10 = icd10.split('.')
    icd10 = ''.join(icd10)
    icd10def = line[3] 
    conf = line[4] 
    selfrckeysnew[selfrc] = meaning
    selfrcdictnew[selfrc,'meaning'] = meaning
    if (selfrc,'icd') in selfrcdictnew:
        selfrcdictnew[selfrc,'icd'].append(icd10)
    else:
        selfrcdictnew[selfrc,'icd'] = [icd10]

In [8]:

# for each self reported coding let's add the list of individuals that belong
# for each icd10 code that also belongs add the list of individuals 
# Gather data from ICD10 code structure
icdcode = ['20002','41202','40001','40002','40006','41204','41270','41201']

#20002 ukb40831.tab
#40001 ukb40831.tab
#40002 ukb40831.tab
#41201 ukb40831.tab
#41202 ukb40831.tab
#41204 ukb40831.tab
#40006 ukb40831.tab
#41270 ukb37855.tab


In [9]:
phenkeys = list(selfrckeysnew.keys())

In [10]:

with open("/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/2007183/40831/download/ukb40831.tab") as selfRpt2:
    header = selfRpt2.readline()
    
header = header.rstrip().split('\t')
idxmap = {}
for idx , item in enumerate(header):
    item = item.split('.')[1]
    if item in icdcode:
        idxmap[idx] = item

In [11]:

        
with open("/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/2007183/40831/download/ukb40831.tab") as ICD102:
    # Skip the header
    next(ICD102)

    nphen = len(phenkeys)
    print(nphen)
    subjectsnew = {}
    g = 0
    for line in ICD102:
        g += 1
        line = line.rstrip()
        line = line.split('\t')
        subject = line[0].strip('"')
        eid = subject
        if g % 100 == 0:
            print(g,eid)
        subjectsnew[eid] = np.ones(len(selfrckeysnew.keys()), dtype=np.int)
        # Check the remaining columns and see if any contain a field code
        for idx, item in enumerate(line):
            if idx == 0:
                continue
            item = item.strip('"')
            # If they contain a field code
            if len(item) > 0 and idx in idxmap and item != "NA":
                if idxmap[idx] == "20002":
                    for k in range(0,nphen):
                        if item == phenkeys[k]:
                            temp = subjectsnew[eid]
                            temp[k] = 2.0
                            subjectsnew[eid] = temp
                        elif item[0:3] == phenkeys[k]:
                            temp = subjectsnew[eid]
                            temp[k] = 2.0
                            subjectsnew[eid] = temp                            
                else:
                    for k in range(0,nphen):
                        if item in selfrcdictnew[phenkeys[k],'icd']:
                            # If there is a match, change subject value to 2
                            temp = subjectsnew[eid]
                            temp[k] = 2.0
                            subjectsnew[eid] = temp
                        elif item[0:3] in selfrcdictnew[phenkeys[k],'icd']:
                            temp = subjectsnew[eid]
                            temp[k] = 2.0 
                            subjectsnew[eid] = temp


446
100 3462576
200 5631482
300 1494083
400 1192637
500 2576266
600 5254524
700 5475086
800 2500788
900 3744770
1000 4038906
1100 1715621
1200 1819277
1300 3197114
1400 3050320
1500 5373925
1600 3362986
1700 5963606
1800 4068092
1900 4098162
2000 2421197
2100 1742004
2200 2316170
2300 1851326
2400 1893835
2500 4530619
2600 1102691
2700 3745448
2800 4370089
2900 3287619
3000 3147084
3100 4457415
3200 2968820
3300 4526884
3400 5780379
3500 5137727
3600 2444461
3700 1972953
3800 1712641
3900 1632701
4000 3212058
4100 5453411
4200 2456066
4300 2724485
4400 3540615
4500 5681492
4600 1000254
4700 2092213
4800 3514090
4900 1081612
5000 2047601
5100 2034966
5200 5084214
5300 4002030
5400 1438588
5500 2328257
5600 2860372
5700 2480785
5800 5796621
5900 5852172
6000 5847026
6100 5173253
6200 3993015
6300 1043269
6400 4779958
6500 4455913
6600 5348212
6700 3563029
6800 4109016
6900 2251671
7000 2640964
7100 4471448
7200 3892445
7300 5711673
7400 5898656
7500 1967722
7600 1729695
7700 1312139
7800

59400 1376969
59500 2052002
59600 2148166
59700 1754390
59800 2761353
59900 4611453
60000 2143864
60100 4907116
60200 4937614
60300 1987193
60400 4272250
60500 3997960
60600 4387979
60700 2339081
60800 1357407
60900 4132924
61000 3163987
61100 5741623
61200 2777653
61300 1688642
61400 1863503
61500 3320658
61600 5140353
61700 3275795
61800 5463021
61900 3058901
62000 4387504
62100 1112707
62200 5677908
62300 2101901
62400 1976813
62500 5888349
62600 5657160
62700 4579406
62800 1340952
62900 1818430
63000 1359023
63100 5954564
63200 1528777
63300 3901933
63400 5750731
63500 5058748
63600 3728272
63700 2102141
63800 4379687
63900 4603626
64000 3256838
64100 2579182
64200 2095427
64300 4706190
64400 5127948
64500 3367879
64600 3689548
64700 5520497
64800 4940263
64900 4171813
65000 2565933
65100 4853113
65200 3095105
65300 3873105
65400 3257546
65500 4740323
65600 5431429
65700 4481100
65800 3832247
65900 1688881
66000 2618282
66100 3392578
66200 5436851
66300 3121010
66400 1483160
66500 

116800 1710441
116900 5256136
117000 4248559
117100 1627708
117200 3733666
117300 5788367
117400 4179940
117500 5398996
117600 5853120
117700 4136599
117800 5034132
117900 1386768
118000 5239393
118100 5927783
118200 4613921
118300 2545512
118400 3497589
118500 2885098
118600 2212447
118700 4182399
118800 2120874
118900 4475188
119000 1638406
119100 3194688
119200 1716257
119300 3196653
119400 1615064
119500 1636186
119600 2213705
119700 3789298
119800 5633609
119900 4786783
120000 4344477
120100 3592218
120200 3701838
120300 1120855
120400 4097020
120500 4814930
120600 2076080
120700 1261029
120800 3287533
120900 4337099
121000 3361919
121100 5709071
121200 3090155
121300 2675537
121400 5207287
121500 1594110
121600 1158196
121700 4258427
121800 2198285
121900 3837912
122000 3017157
122100 2906629
122200 1545151
122300 5179910
122400 1340247
122500 3958342
122600 2677144
122700 1289157
122800 5603727
122900 5859683
123000 4987798
123100 2080460
123200 3186126
123300 3761728
123400 433

171500 4324138
171600 2407939
171700 4821488
171800 1107821
171900 4668417
172000 3066062
172100 2632801
172200 2362340
172300 4152632
172400 3285253
172500 3206137
172600 1567556
172700 5026257
172800 3721705
172900 3865927
173000 1526974
173100 2871852
173200 4510284
173300 1694339
173400 1766563
173500 4232806
173600 2700531
173700 4874346
173800 5962931
173900 4465567
174000 3500250
174100 1378723
174200 5497783
174300 1953678
174400 3497640
174500 2431720
174600 3598295
174700 2644739
174800 4397742
174900 2504745
175000 3901678
175100 5327124
175200 5260495
175300 4667959
175400 3097345
175500 5380002
175600 2018092
175700 1536684
175800 2666490
175900 5849760
176000 4835651
176100 1879811
176200 4601933
176300 3800288
176400 2678141
176500 3092736
176600 1481805
176700 1271058
176800 3988611
176900 3846551
177000 1326618
177100 5590383
177200 4247006
177300 4010150
177400 2093215
177500 5353586
177600 4982565
177700 4069341
177800 2065900
177900 2456545
178000 3241705
178100 276

226200 4191861
226300 1858836
226400 5014559
226500 4700876
226600 2628860
226700 2962240
226800 3220748
226900 2064325
227000 5433921
227100 2881588
227200 1576859
227300 4368908
227400 5532547
227500 2356373
227600 4177090
227700 2487064
227800 1093770
227900 4236138
228000 5260852
228100 3442029
228200 5419474
228300 5848597
228400 1319771
228500 3975937
228600 1492048
228700 5442564
228800 1772780
228900 5280795
229000 5522116
229100 4015172
229200 2849391
229300 4091181
229400 1710017
229500 3129386
229600 5940284
229700 1831767
229800 4521007
229900 4375332
230000 1499033
230100 3249639
230200 4899838
230300 4807991
230400 1071500
230500 1259068
230600 5964255
230700 2963869
230800 4264927
230900 4175532
231000 1856790
231100 5340933
231200 5695352
231300 3518680
231400 4752679
231500 1839224
231600 5081994
231700 2604859
231800 5516548
231900 1222360
232000 3023381
232100 3057210
232200 2420270
232300 3363526
232400 2191562
232500 5314679
232600 2724351
232700 5727858
232800 188

280900 5489263
281000 2712922
281100 2293398
281200 5368730
281300 4278765
281400 5674956
281500 2914544
281600 4369651
281700 3994414
281800 3577680
281900 2281891
282000 1645187
282100 5393666
282200 3145927
282300 2270973
282400 2161784
282500 5012067
282600 3763848
282700 6021024
282800 1035964
282900 3704050
283000 5008192
283100 5450346
283200 1595987
283300 3398031
283400 5074714
283500 3266687
283600 2010872
283700 4097354
283800 3918407
283900 4003414
284000 2434001
284100 2738471
284200 5372267
284300 1563428
284400 2182814
284500 4646850
284600 3883025
284700 3046012
284800 3167145
284900 4122525
285000 2212772
285100 3342713
285200 3992568
285300 4618296
285400 1383789
285500 5865160
285600 5985005
285700 2803705
285800 1608354
285900 1726368
286000 1920653
286100 2077620
286200 4670144
286300 1021376
286400 5959519
286500 4152957
286600 1818260
286700 2512331
286800 2333159
286900 5222488
287000 1271382
287100 3361608
287200 4299835
287300 4704671
287400 2931218
287500 103

335600 5385025
335700 5354154
335800 4429567
335900 3243715
336000 3361743
336100 4341958
336200 5208850
336300 3008191
336400 1072977
336500 4567759
336600 2031411
336700 4802147
336800 4113383
336900 1515248
337000 1261334
337100 1299579
337200 5149431
337300 1554973
337400 4054662
337500 3446239
337600 3767028
337700 2938966
337800 5949839
337900 1623503
338000 2823347
338100 1512691
338200 3199825
338300 2222178
338400 3832218
338500 4510932
338600 3292059
338700 3101944
338800 6025805
338900 3587789
339000 4023303
339100 1005271
339200 2331803
339300 2283196
339400 3581170
339500 1428143
339600 5611389
339700 5669792
339800 2726261
339900 1399464
340000 1198907
340100 3995588
340200 3930296
340300 4190540
340400 4132989
340500 4666363
340600 4134294
340700 5818907
340800 1200938
340900 2936691
341000 3462948
341100 1225006
341200 2263662
341300 1878413
341400 1195664
341500 1014199
341600 4989136
341700 2558611
341800 1771771
341900 3714144
342000 3701329
342100 4456268
342200 366

390300 4491378
390400 3203345
390500 3528829
390600 5294890
390700 4018417
390800 3390395
390900 5267689
391000 5075685
391100 5579227
391200 1537149
391300 5400759
391400 2289439
391500 1313702
391600 2169788
391700 3288960
391800 4294590
391900 1236517
392000 2416148
392100 4675349
392200 4073631
392300 1249348
392400 4145437
392500 2722007
392600 1919931
392700 1256330
392800 1025931
392900 2556104
393000 5792608
393100 3881515
393200 4638634
393300 2346395
393400 1095971
393500 5355581
393600 3893523
393700 1371700
393800 3001200
393900 5341140
394000 3253666
394100 2300294
394200 1087424
394300 2206628
394400 3003565
394500 2822125
394600 1391274
394700 1958739
394800 5082030
394900 2829406
395000 4693820
395100 4428464
395200 5922818
395300 5945184
395400 3941255
395500 1350967
395600 2648094
395700 3020401
395800 5284232
395900 2317768
396000 4933926
396100 3441836
396200 2057929
396300 3721268
396400 1512388
396500 3895328
396600 5432243
396700 2455148
396800 1677552
396900 163

445000 2639409
445100 4655963
445200 2996398
445300 3423606
445400 1183055
445500 1633432
445600 3791740
445700 2427193
445800 3670483
445900 2515441
446000 4009116
446100 4600384
446200 1453173
446300 3677241
446400 5357889
446500 2736464
446600 2037627
446700 4041058
446800 4243344
446900 4577151
447000 5858426
447100 1483197
447200 5023918
447300 2328665
447400 2147411
447500 2073569
447600 4541590
447700 3940350
447800 2218195
447900 2579700
448000 2095337
448100 5405586
448200 1016888
448300 2228397
448400 4761114
448500 4748380
448600 3973243
448700 1454401
448800 1513721
448900 3997172
449000 4007734
449100 4979215
449200 4596849
449300 2322865
449400 4354887
449500 3723161
449600 5334275
449700 4081928
449800 3130822
449900 5821015
450000 1194491
450100 5599053
450200 5802031
450300 5477056
450400 2109723
450500 2066509
450600 1024349
450700 2248775
450800 4551975
450900 5973790
451000 3914847
451100 4961097
451200 3349073
451300 5213613
451400 3159821
451500 5852189
451600 436

499700 1901173
499800 4201156
499900 3494809
500000 5637842
500100 1224389
500200 4697092
500300 3317345
500400 2587173
500500 5727177
500600 2850587
500700 5656061
500800 5823202
500900 5542389
501000 5630754
501100 1207788
501200 5151333
501300 2654374
501400 5995936
501500 1317073
501600 1260784
501700 2468259
501800 3053306
501900 2678872
502000 2384030
502100 2947170
502200 5698389
502300 5221169
502400 5769266
502500 1995151


In [12]:
#41270 data field read in
with open("/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/2005693/37855/download/ukb37855.tab") as selfRpt2:
    header = selfRpt2.readline()
    
header = header.rstrip().split('\t')
idxmap = {}
for idx , item in enumerate(header):
    item = item.split('.')[1]
    if item in icdcode:
        idxmap[idx] = item

In [13]:
with open("/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/2005693/37855/download/ukb37855.tab") as ICD102:
    # Skip the header
    next(ICD102)

    nphen = len(phenkeys)
    print(nphen)
    g = 0
    for line in ICD102:
        g += 1
        line = line.rstrip()
        line = line.split('\t')
        subject = line[0].strip('"')
        eid = subject
        if g % 100 == 0:
            print(g,eid)
        #subjectsnew[eid] = np.ones(len(selfrckeysnew.keys()), dtype=np.int)
        # Check the remaining columns and see if any contain a field code
        for idx, item in enumerate(line):
            if idx == 0:
                continue
            item = item.strip('"')
            # If they contain a field code
            if len(item) > 0 and idx in idxmap:
                if idxmap[idx] == "41270":
                    for k in range(0,nphen):
                        if item in selfrcdictnew[phenkeys[k],'icd']:
                            # If there is a match, change subject value to 2
                            temp = subjectsnew[eid]
                            temp[k] = 2.0
                            subjectsnew[eid] = temp
                        elif item[0:3] in selfrcdictnew[phenkeys[k],'icd']:
                            temp = subjectsnew[eid]
                            temp[k] = 2.0 
                            subjectsnew[eid] = temp


446
100 3924736
200 5340450
300 3245056
400 2225686
500 2228253
600 4283036
700 5626452
800 5214843
900 5095178
1000 1851900
1100 2717078
1200 5713317
1300 4830539
1400 5203290
1500 2529713
1600 4885528
1700 1817849
1800 1987922
1900 4684908
2000 3310014
2100 3479831
2200 2076351
2300 1031124
2400 5274566
2500 1221076
2600 4834082
2700 4833283
2800 5470091
2900 2710080
3000 5215335
3100 1754326
3200 4115022
3300 4521697
3400 4666082
3500 5162031
3600 3540705
3700 1795700
3800 3354112
3900 3355986
4000 3449973
4100 4974211
4200 5487473
4300 5513395
4400 3852091
4500 3106626
4600 2786370
4700 2525387
4800 5288566
4900 2356108
5000 5963282
5100 5480982
5200 1491124
5300 4664879
5400 4571984
5500 3353472
5600 2547372
5700 4818244
5800 1395181
5900 2109117
6000 1414266
6100 2133861
6200 1205373
6300 3147041
6400 1001486
6500 4169409
6600 4275812
6700 4099476
6800 3739714
6900 1013998
7000 4664894
7100 4922624
7200 1379912
7300 5943737
7400 4207762
7500 2229373
7600 3545683
7700 3823106
7800

59400 2558817
59500 5397387
59600 3989348
59700 4032880
59800 1910009
59900 5610536
60000 4970187
60100 5745182
60200 5888505
60300 4437816
60400 2661351
60500 1242835
60600 5071726
60700 1333701
60800 4787005
60900 4158327
61000 2129011
61100 5308059
61200 3144852
61300 2218486
61400 1289755
61500 2771907
61600 1465716
61700 4439805
61800 5786259
61900 1120609
62000 3961813
62100 2984516
62200 1473741
62300 1929295
62400 2770967
62500 5579552
62600 1644759
62700 5115172
62800 4245439
62900 1535679
63000 1331143
63100 2758342
63200 1792661
63300 4499156
63400 1717986
63500 3621031
63600 3970387
63700 5367728
63800 2178911
63900 1868151
64000 5811164
64100 5090856
64200 3936767
64300 3460287
64400 4398282
64500 5957253
64600 5267278
64700 5299919
64800 1643776
64900 5734152
65000 4294491
65100 3985320
65200 5600650
65300 1250229
65400 5178527
65500 4096710
65600 1370725
65700 3821007
65800 5985005
65900 2685855
66000 2027840
66100 3108728
66200 4909994
66300 1795281
66400 2367694
66500 

116800 5752123
116900 5582669
117000 3367714
117100 2085810
117200 1583427
117300 2125475
117400 3504979
117500 1159313
117600 5981971
117700 2847841
117800 2797563
117900 5880866
118000 4909504
118100 1782979
118200 3765859
118300 2776165
118400 2319679
118500 1584650
118600 3762653
118700 2724724
118800 4118102
118900 2693109
119000 1071284
119100 2160563
119200 5135080
119300 4227547
119400 4725007
119500 2835671
119600 3372705
119700 3029352
119800 1924482
119900 5009424
120000 4594094
120100 5071624
120200 3386666
120300 5361828
120400 1801519
120500 4217637
120600 5729252
120700 1967089
120800 2408110
120900 5391332
121000 5565757
121100 3535148
121200 4165634
121300 2012989
121400 5417446
121500 4749133
121600 1335066
121700 1697240
121800 1283971
121900 1546819
122000 5128717
122100 2674242
122200 5138541
122300 2489612
122400 3661506
122500 4424321
122600 1832614
122700 3190439
122800 4438173
122900 2082486
123000 5965091
123100 5942272
123200 1914233
123300 5867598
123400 536

KeyError: '3011697'

In [14]:
# Write out phenotype matrix

# Print the subject phenotype matrix to file:
with open("20_03_27_highconfidenceqc.csv", "w+") as outFile:
    outFile.write("eid" + "," + ",".join([selfrcdictnew[phenkeys[i],'meaning'] for i in range(0,len(phenkeys))]) + "\n")
    for subject in subjectsnew:
        outFile.write(subject + "," +",".join(map(str,subjectsnew[subject])) + "\n")
        


In [20]:
import os

In [21]:
for i in range(len(phenkeys)):
    os.system("awk -F ',' 'NR>1 {print($1,$1, $"+str(i+2)+")}' 20_03_27_highconfidenceqc.csv > HC"+str(i)+".phe")

In [15]:
hcMat = pd.read_csv("20_03_27_highconfidenceqc.csv", index_col=0)

In [16]:
# Count cases for each column
counts = hcMat[hcMat > 1].sum()

In [18]:
# Print family history+ICD10+self-reported counts to file
sorted_counts = counts.sort_values(ascending=False)
print(counts.sort_values(ascending=False))
outFile=open("20_03_27_highconfidence_summary.csv", "w+")
outFile.write("field name,count\n")
for index, value in sorted_counts.iteritems():
    outFile.write(",".join(map(str,[index,int(value)/2]))+"\n")
outFile.close()

hypertension              343718.0
essential hypertension    229196.0
hiatus hernia             139350.0
asthma                    135690.0
high cholesterol          134626.0
                            ...   
yellow fever                  32.0
pancytopenia                  20.0
varicella zoster virus        16.0
benign insulinoma              8.0
hepatitis d                    2.0
Length: 446, dtype: float64
